In [1]:
from scipy.optimize import linprog
import numpy as np
from itertools import combinations 

In [2]:
distance_mat=np.loadtxt('cosine_sim_matrix_proj_researcher_2014.txt')

In [3]:
real_project=np.loadtxt('discipline_vectors.txt')

# Wasserstein distance

In [4]:
# Distance matrix between disciplines
M = np.array([[0, 9, 7, 2, 8],
              [9, 0, 8, 10, 3],
              [7, 8, 0, 6, 5],
              [2, 10, 6, 0, 9],
              [8, 3, 5, 9, 0]])
M

array([[ 0,  9,  7,  2,  8],
       [ 9,  0,  8, 10,  3],
       [ 7,  8,  0,  6,  5],
       [ 2, 10,  6,  0,  9],
       [ 8,  3,  5,  9,  0]])

In [5]:
p1=np.array([0.3,0.4,0.3,0,0])
p2=np.array([0,0,0,0.5,0.5])
p3=np.array([0,0,1,0,0])
p4=np.array([0,1,0,0,0])
p5=np.array([0.2,0.2,0.2,0.4,0])
p6=np.array([0,0,0,0.6,0.4])

In [6]:
people=np.array([p1, p2, p3, p4, p5, p6])

In [7]:
# Transform sparse arrays p,q and M to dense arrays by deleting entries (disciplines) which are zero for both p and q.
def discipline_compressor(p,q,M):

    zero_entries=[]

    for i in range(len(p)):
        if p[i]==0 and q[i]==0:
            zero_entries.append(i)
            
    zero_entries.sort(reverse=True)
    p_dense=p.copy()
    q_dense=q.copy()
    M_dense=M.copy()

    for j in zero_entries:
        M_dense=np.delete(M_dense, j, 1)
        M_dense=np.delete(M_dense, j, 0)
        p_dense= np.delete(p_dense,j)
        q_dense= np.delete(q_dense,j)
    
    return p_dense,q_dense,M_dense

In [8]:
def constraint_maker(n):
    constraint_matrix=np.zeros((2*n,n**2))

    for i in range(n):
        constraint_matrix[i][i*n:i*n+n]=np.ones(n)
        for j in range(n):
            constraint_matrix[i+n][n*j+i]=1

    constraint_matrix=np.delete(constraint_matrix,0,0) #delete first (redundent) row to get full row rank
    return constraint_matrix

In [9]:
def wasserstein(p,q,M):
    p_dense,q_dense,M_dense=discipline_compressor(p,q,M)

    obj = M_dense.flatten()


    lhs_eq = constraint_maker(len(p_dense))  
    rhs_eq = np.append(p_dense,q_dense)[1:]      

    opt = linprog(c=obj, 
                  A_eq=lhs_eq, b_eq=rhs_eq, 
                  method="revised simplex")
    return opt.fun


In [10]:
wasserstein(p6,p2, M)

0.8999999999999998

# Diversity of a project

## Rao Stirling index

In [11]:
def unique_person_project(Project):
    freq_list=[]
    for unique in np.unique(Project, axis=0):
        count=0
        for pers in Project:   
            if all(unique == pers):
                count=count+1

        freq_list=freq_list+[(unique,count)]
    return freq_list

In [12]:
def RaoStirling(Project,M):
    freq_list=unique_person_project(Project)
    size=len(Project)
    comb = list(combinations(freq_list, 2))
    som=0
    for i in list(comb): 
        freq1=i[0][1]/size
        freq2=i[1][1]/size
        som=som+wasserstein(i[0][0],i[1][0],M)*freq1*freq2
    return som


In [13]:
P=np.array([p2, p6, p1, p4, p1, p3, p1, p2, p5, p5])

In [14]:
unique_person_project(P)

[(array([0. , 0. , 0. , 0.5, 0.5]), 2),
 (array([0. , 0. , 0. , 0.6, 0.4]), 1),
 (array([0., 0., 1., 0., 0.]), 1),
 (array([0., 1., 0., 0., 0.]), 1),
 (array([0.2, 0.2, 0.2, 0.4, 0. ]), 2),
 (array([0.3, 0.4, 0.3, 0. , 0. ]), 3)]

In [15]:
RaoStirling(real_project,distance_mat)

0.21293941145588044

## Average distance to center of mass (ADCM) (only for size(project)<4)

Calculate coordinates of each person of the project from distance matrix using eigenvalue decomposition (https://math.stackexchange.com/questions/156161/finding-the-coordinates-of-points-from-distance-matrix)

In [16]:
P=np.array([p2, p1, p4, p1, p3, p1, p2,p5, np.array([0.1,0.2,0.3,0.4,0])])

In [17]:
def make_distance_matrix(Project,M):
    P_unique=unique_person_project(Project)
    n=len(P_unique)
    D=np.zeros((n,n))

    for i in range(n):
        for j in range(n):
            D[i,j]=wasserstein(P_unique[i][0],P_unique[j][0],M)
            D[j,i]=D[i,j]
    return D

In [18]:
def people_coordinates(Project, M):
    P_unique=unique_person_project(Project)
    n=len(P_unique)

    A=np.zeros((n,n))
    D=make_distance_matrix(Project,M)
    for i in range(n):
        for j in range(n):
            A[i,j]=((D[0,j]**2)+(D[i,0]**2)-(D[i,j]**2))/2

    lam, Q=np.linalg.eigh(A)
    Lam=np.diag(lam)
    QT=Q.transpose()
    np.testing.assert_allclose(QT@Q, np.identity(n), rtol=1e-5, atol=1e-5)
    return Q@Lam**(1/2), lam, A,D

## ADCM 2.0

In [19]:
def ADC(Project,M):
    centroid=np.sum(Project,axis=0)/len(Project)
    som=0
    for person in Project:
        som=som+wasserstein(person, centroid,M)
    return som/len(Project)

In [20]:
def ADM(Project,M):
    smallest_som=-1
    for possible_medoid in Project:
        som=0
        for person in Project:
            som=som+wasserstein(person, possible_medoid,M)
        if smallest_som==-1:
            smallest_som=som
        elif som<smallest_som:
            smallest_som=som
    
    return smallest_som/len(Project)

In [21]:
import random 
def random_project(people): 
    r1 = random.randint(1, 10)
    randomar=np.random.randint(len(people), size=r1)
    P=[]
    for i in randomar:
        P=P+[people[i]]
    return np.array(P)

In [22]:
import pandas as pd
df = pd.DataFrame([], columns=['Project size','ADM','ADC','RaoStirling','ADC/RaoStirling'])
for i in range(50):
    P=random_project(people)
    df2 = pd.DataFrame([[len(P),ADM(P,M),ADC(P,M),RaoStirling(P,M),ADC(P,M)/RaoStirling(P,M)]], columns=['Project size','ADM','ADC','RaoStirling','ADC/RaoStirling'])
    df=df.append(df2, ignore_index=True)
df

<ipython-input-22-bdd65d4f01bc>:5: RuntimeWarning: invalid value encountered in double_scalars
  df2 = pd.DataFrame([[len(P),ADM(P,M),ADC(P,M),RaoStirling(P,M),ADC(P,M)/RaoStirling(P,M)]], columns=['Project size','ADM','ADC','RaoStirling','ADC/RaoStirling'])


,Project size,ADM,ADC,RaoStirling,ADC/RaoStirling
0,2,1.750000,1.750000,0.875000,2.000000
1,7,3.500000,3.402041,2.065306,1.647233
2,5,2.680000,2.592000,1.504000,1.723404
3,8,3.712500,3.696875,2.173437,1.700935
4,3,0.300000,0.400000,0.200000,2.000000
5,1,0.000000,0.000000,0.000000,NaN
6,7,3.500000,3.402041,2.065306,1.647233
7,1,0.000000,0.000000,0.000000,NaN
8,9,3.155556,3.024691,1.916049,1.578608
9,7,3.242857,3.232653,1.967347,1.643154
